In [287]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, LongType, MapType, Row
from pyspark.sql.functions import *
from pyspark.sql import Window
import numpy as np
import pandas as pd
import sys

spark = SparkSession.builder.appName("Auto Report–Spark").getOrCreate()



# 1. Load Data

In [288]:
schema = StructType([ \
    StructField('incident_id', IntegerType(), True), 
    StructField('incident_type', StringType(), True),
    StructField('vin_num', StringType(), True),
    StructField('make', StringType(), True),
    StructField('model', StringType(), True),
    StructField('year', StringType(), True),
    StructField('incident_date', DateType(), True),
    StructField('desc', StringType(), True)
])


In [289]:
df = spark.read.format("csv") \
    .option("header", False) \
    .schema(schema) \
    .load("/Users/mallory/Desktop/DataEngineering/Springboard/DistComp/hadoop_auto/data.csv")
df.show()
#df.cache()

+-----------+-------------+-----------------+--------+------+----+-------------+--------------------+
|incident_id|incident_type|          vin_num|    make| model|year|incident_date|                desc|
+-----------+-------------+-----------------+--------+------+----+-------------+--------------------+
|          1|            I|VXIO456XLBB630221|  Nissan|Altima|2003|   2002-05-08|Initial sales fro...|
|          2|            I|INU45KIOOPA343980|Mercedes|  C300|2015|   2014-01-01|Sold from EuroMotors|
|          3|            A|VXIO456XLBB630221|    null|  null|null|   2014-07-02|   Head on collision|
|          4|            R|VXIO456XLBB630221|    null|  null|null|   2014-08-05| Repair transmission|
|          5|            I|VOME254OOXW344325|Mercedes|  E350|2015|   2014-02-01|    Sold from Carmax|
|          6|            R|VOME254OOXW344325|    null|  null|null|   2015-02-06|Wheel allignment ...|
|          7|            R|VXIO456XLBB630221|    null|  null|null|   2015-01-01|Re

## 1.1: Perform Map Operation 
Implement `extract_key_vin_value()` method

In [290]:
def extract_key_vin_value(x):
    """:param x: data source loaded into SparkSession,
        :output: dictionary tuple with mapping values to be transformed into MapType"""

    vin_number = x.vin_num
    make = x.make
    year = x.year
    model = x.model
    incident_id = x.incident_id
    incident_type = x.incident_type
    incident_date = x.incident_date
    desc = x.desc
  
    return (vin_number, {"make": make, "year": year, "model": model, "incident_id":incident_id, "incident_type":incident_type, "incident_date":incident_date, "desc":desc})


In [291]:
vin_kv = df.rdd.map(lambda x: extract_key_vin_value(x))
# vin_kv.cache()
type(vin_kv)

pyspark.rdd.PipelinedRDD

In [430]:
# QA
# vin_kv.collect()

In [293]:
# QA - was checking how to access each member 😅
# for key, value in vin_kv.collect():
#     # print(key, value)
#     print(key, value["make"], value["year"], value["model"])

## 1.2 Perform Group Aggregation to Populate Make & Year to All Records
Implement `populate_make()` method

In [431]:
# vin_kv.collectAsMap()
sc = spark.sparkContext
def populate_make(data_rdd):
    # data = data_rdd.collect()
    print(type(data_rdd))

    output = []

    for member in data_rdd:
        value = member[1]
        # print(member[1])
        key = member[0]
        incident_id = value["incident_id"]
        incident_type = value["incident_type"]
        incident_date = value["incident_date"]
        desc = value["desc"]
        if value["make"] != None:
                make = value["make"]
        if value["year"] != None:
                year = value["year"]
        if value["model"] != None:
                model = value["model"]
        output.append({key: {"make": make, "year": year, "model": model, "incident_id":incident_id, "incident_type":incident_type, "incident_date":incident_date, "desc": desc}})
    #return sc.parallelize(output)
    return output


In [433]:
# QA
#print(populate_make(vin_kv))
#populate_make(vin_kv.collect())
# type(x)
#type(populate_make(vin_kv.collect()))

In [296]:
# # creating a bridge table to collect master information for each make and model
# bridge_schema = StructType([
#     StructField("vin_key", StringType(), True),
#     # use MapType to make use of key-value pairs returned by function
#     StructField("properties", MapType(StringType(), StringType(), True))
# ])

# bridge_df = spark.createDataFrame(data=vin_kv, schema=bridge_schema)
# bridge_df.show(truncate=False)
# # check schema
# bridge_df.printSchema()

# # options here were map_concat(), coalesce(), and explode()
# # explode map column to create a new row for each element in the given map column
# bridge_df = bridge_df.select("vin_key", explode("properties"))

# bridge_df.printSchema()  # schema will validate the explosion
# bridge_df.show()

# # get distinct values & drop null to avoid duplication
# df_distinct = bridge_df.select(
#     "vin_key", "key", "value").distinct().na.drop().sort("key")
# df_distinct.show(truncate=False)

# # get rid of null values to get year make and model as an array of maps
# map_df = df_distinct.select("vin_key", create_map("key", "value").alias("map")) \
#     .groupBy("vin_key") \
#     .agg(collect_list("map").alias("make_model_year")) \
#     # .cache()

# # make = map_df.select("vin_key", map_df.make_model_year[0].alias("make")).show()

# # model = map_df.select("vin_key", map_df.make_model_year[1].alias("model")).show()

# # year = map_df.select("vin_key", map_df.make_model_year[2].alias("year")).show()

# # map_df.show(truncate=False)
# # map_df.printSchema()

# map_df = map_df.select("vin_key", map_df.make_model_year[0].alias("make_map"), map_df.make_model_year[1].alias(
#     "model_map"), map_df.make_model_year[2].alias("year_map"))  # .cache()

# map_df.show()

# map_df = map_df.select("vin_key", map_df.make_map.getItem("make").alias("make"),
#                        map_df.model_map.getItem("model").alias("model"),
#                        map_df.year_map.getItem("year").alias("year")) \
#     .show()


In [434]:
# enhance_make = vin_kv.groupByKey().flatMap(lambda kv: populate_make(kv[1]))
enhance_make = vin_kv.groupByKey().flatMap(lambda kv: populate_make(kv))
type(enhance_make)

pyspark.rdd.PipelinedRDD

# 2. Count the # of Accident Occurrences for Each Vehicle Make & Year
## 2.1 Perform Map Operation
Implement `extract_key_make_value()` method

In [435]:
def extract_key_make_value(data):
    #data = sc.parallelize(data)
    #data = data.toLocalIterator()
    #print(data)
    # print(hasattr(data, "collect"))
    print(type(data))

    for i in data.take(100):
        print(i)
    # for item in data:
    #     print(hasattr(data, "collect"))
    #     #data.collect()
    #     if item[1]["incident_type"] == 'A':
    #         val = ((item[1]["make"],item[1]["year"]), 1)
    #         return val
    #     else:
    #         val = ((item[1]["make"],item[1]["year"]), 0)
    #     return val

extract_key_make_value(enhance_make)


<class 'pyspark.rdd.PipelinedRDD'>


<class 'tuple'>
21/11/02 18:44:26 ERROR Executor: Exception in task 0.0 in stage 275.0 (TID 218)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/mallory/Desktop/DataEngineering/Springboard/DistComp/kafka-mini/env/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/Users/mallory/Desktop/DataEngineering/Springboard/DistComp/kafka-mini/env/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/mallory/Desktop/DataEngineering/Springboard/DistComp/kafka-mini/env/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/mallory/Desktop/DataEngineering/Springboard/DistComp/kafka-mini/env/lib/python3.9/site-packages/pyspark/rdd.py", line 1562, in takeUpToNumLeft
 

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 275.0 failed 1 times, most recent failure: Lost task 0.0 in stage 275.0 (TID 218) (10.0.0.66 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/mallory/Desktop/DataEngineering/Springboard/DistComp/kafka-mini/env/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/Users/mallory/Desktop/DataEngineering/Springboard/DistComp/kafka-mini/env/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/mallory/Desktop/DataEngineering/Springboard/DistComp/kafka-mini/env/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(

21/11/03 01:16:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1041683 ms exceeds timeout 120000 ms
21/11/03 01:16:44 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
make_kv = enhance_make.map(lambda x: extract_make_key_value(x))
type(make_kv)
print(make_kv)